In [1]:
import numpy as np
import torch
import collections
import json
import transformers
from transformers import pipeline, set_seed

tokenizer = transformers.AutoTokenizer.from_pretrained('gpt2')
lm = transformers.AutoModelForCausalLM.from_pretrained('gpt2')

/Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/torchvision/image.so, 6): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: /Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/torchvision/image.so
  Expected in: /Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib
 in /Users/annhe/anaconda3/envs/q_star/lib/python3.9/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong wit

In [22]:
tokens = tokenizer("Hello ", return_tensors='pt')
print(tokens)

{'input_ids': tensor([[15496,   220]]), 'attention_mask': tensor([[1, 1]])}


In [23]:
k = 5
beam_output = lm.generate(
    tokens['input_ids'],
    max_new_tokens=1,
    num_beams=k,
    num_return_sequences=k,
    return_dict_in_generate=True,
    output_logits=True,
    output_scores=True,
    early_stopping=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [24]:
print(beam_output.sequences_scores)

# how sequences_scores are computed
# https://discuss.huggingface.co/t/announcement-generation-get-probabilities-for-generated-output/30075

tensor([-2.0736, -2.3311, -3.0310, -3.0803, -3.3449])


In [25]:
# testing logits_to_token_strings

print(beam_output.keys())

odict_keys(['sequences', 'sequences_scores', 'scores', 'logits', 'beam_indices', 'past_key_values'])


In [26]:
print(beam_output.logits[0].shape)

torch.Size([5, 50257])


In [46]:
def logits_to_token_strings(logits):
    print(logits.shape)
    next_tokens = torch.log_softmax(beam_output.logits[0], dim=1)
    next_tokens = torch.argmax(next_tokens, dim=1)
    str_repr = tokenizer.batch_decode(next_tokens)
    next_tokens = list(torch.chunk(next_tokens,chunks=5,dim=0))
    return next_tokens, str_repr

In [47]:
next_tokens, str_repr = logits_to_token_strings(beam_output.logits[0])
print(str_repr)

torch.Size([5, 50257])
tensor([2474, 2474, 2474, 2474, 2474])
['!"', '!"', '!"', '!"', '!"']


In [45]:
print(next_tokens)

[tensor([2474]), tensor([2474]), tensor([2474]), tensor([2474]), tensor([2474])]
